In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from time import time
import os
os.environ['BACKEND'] = 'tf'  # or tf or comment for auto
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
from unisim import TextSim

Loaded backend
Using tf with cpu


2023-10-30 19:30:17.065535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-30 19:30:17.065601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 6ed3797090ef
2023-10-30 19:30:17.065608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 6ed3797090ef
2023-10-30 19:30:17.065728: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.10
2023-10-30 19:30:17.065762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.10
2023-10-30 19:30:17.065767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.10
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and i

In [5]:
tsim = TextSim(index_type="exact")

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


UniSim is storing a copy of the indexed data
If you are using large data corpus, consider disabling this behavior using store_data=False
Accelerator is not available, using cpu instead


In [35]:
emb = tsim.embed(["a"*512 + "b"*512, "test"])[0]

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [38]:
import numpy  as np
norm = np.linalg.norm(emb, axis=1)

In [39]:
norm.shape

(2,)

In [43]:
norm = np.expand_dims(norm, axis=1)

In [45]:
norm.shape

(2, 1)

In [44]:
emb = emb/norm

In [47]:
emb.shape

(2, 256)

In [48]:
import tensorflow as tf
tf.matmul(emb, emb, transpose_b=True)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.        , 0.3017627 ],
       [0.3017627 , 0.99999976]], dtype=float32)>

In [36]:
# tsim.embed(["a"*512 + "b"*512, "a"*512 + "b"*512])[0].shape

(2, 256)

In [37]:
tsim.similarity("a"*512 + "b"*512, "a"*512 + "b"*512)

[[-5.27383238e-02 -1.14388177e-02  9.93686542e-02  3.92375663e-02
  -3.86192016e-02  4.26479522e-03  1.65452827e-02 -5.77455480e-03
  -2.77983677e-03  1.37475342e-03  1.90549884e-02  1.00033730e-03
   3.73940021e-02 -1.13019478e-02 -5.33627719e-02 -5.23865633e-02
   2.05809344e-02  1.47199407e-02 -7.49011189e-02  7.06552714e-02
   8.64187032e-02  6.35379851e-02  6.78981394e-02 -1.22719537e-03
   2.23861411e-02 -5.02633937e-02  3.05270553e-02  1.52998129e-02
  -5.12180589e-02  2.59483811e-02  7.16330484e-02  2.12619733e-02
  -5.37315756e-02 -2.27460675e-02 -2.82430602e-03  7.89352506e-02
  -1.00233741e-02  7.94798955e-02  2.88114399e-02  1.95103846e-02
   1.68024115e-02  8.21648911e-02 -9.70574021e-02  2.12170891e-02
   1.24121137e-01  1.94037445e-02 -1.10029019e-02 -1.48957968e-03
  -3.82030457e-02  2.90632825e-02  8.63523483e-02 -5.51700890e-02
   6.60778880e-02 -6.35483265e-02  1.39834564e-02  1.03362560e-01
  -3.35898697e-02  3.11839990e-02  1.09014930e-02 -8.98631290e-04
  -1.30234

0.7407481670379639

In [22]:
tsim.embed(["test" * 10000, "test"])[0]

array([[-5.03027476e-02,  4.86846007e-02, -3.12375110e-02,
         2.81484183e-02,  1.59302223e-02,  7.01260120e-02,
         4.96594310e-02, -4.32223603e-02,  8.52845907e-02,
         8.45154524e-02,  3.70270386e-02,  2.84668580e-02,
        -7.11765140e-02, -2.49430891e-02,  2.48743314e-02,
         4.16290527e-03, -1.12633156e-02, -1.89479683e-02,
        -3.19192633e-02,  6.70388639e-02, -8.96873102e-02,
         1.28505692e-01,  1.19441047e-01, -4.38625775e-02,
         6.79713637e-02, -5.91345318e-02, -7.68526420e-02,
         9.24366154e-03, -2.45709717e-02, -2.40435898e-02,
        -7.24257976e-02, -7.78404549e-02, -4.53997329e-02,
        -2.74638855e-03,  8.77560303e-02, -1.42851565e-02,
         1.03717692e-01,  7.72972479e-02,  2.17068549e-02,
        -3.21752951e-02,  5.99052534e-02,  9.00297314e-02,
        -1.54160354e-02,  6.37951270e-02, -2.21439563e-02,
        -7.88561627e-03, -2.88959891e-02, -3.68120894e-02,
        -1.14372693e-01,  1.82515346e-02,  2.98967417e-0

In [4]:
apa = 'apa' * 1024
g, e = tsim.embed(['2 strings one family'])
print(g.shape, len(e))
g, e = tsim.embed(['2 strings one family', apa])
print(g.shape, len(e))

(1, 256) 1
(2, 256) 2


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
N = 100
ts = time()
for _ in range(N):
    tsim.embed(['2 strings one family'])
total = (time() - ts) * 1000
print(f'total ms {round(total)} - {round(total/N)}ms')

total ms 534 - 5ms


In [6]:
t = 'apa' * 1024
d = tsim.similarity('apapa', t)
print(d)

# golden test 0.48
d = tsim.similarity('Doubting dreams'.lower(), 'rough winds do shake the darling buds of may,'.lower())
print('golden', d)

d = tsim.similarity('I love icecreams', 'I love icecreams')
print('exact', d)

0.2681187093257904
golden 0.48520326614379883
exact 0.9999998807907104


In [7]:
example = 't' * 4096
print(len(example))


4096


In [25]:
tsim.reset_index()
ts  = time()
chunk = 'cookies'
partial =  'k' * 530 + chunk
tsim.add([example,  partial,  'I love icecreams', 'tea oh tea', 'cookiess!!!', 'no cookies for you'])
tsim.add(['icecream or icecream sandwich that is the question'])

rc = tsim.search(['I love icecreams', 'my icecream', 'cookiess!!!', chunk], pk=5, gk=5)
total = (time() - ts) * 1000
print(f'total ms {round(total)}')
print(rc)
tsim.visualize(rc.results[3])

total ms 122
-[Results Statistics]-
Number of Results: 4
Total Global Matches: 2
Total Partial Matches: 3
Query 3: "cookies"
  idx  is_global      global_sim  is_partial      partial_sim  content
-----  -----------  ------------  ------------  -------------  --------------------------------
    4  False                0.89  True                   0.89  cookiess!!!
    5  False                0.8   False                  0.8   no cookies for you
    2  False                0.44  False                  0.44  I love icecreams
    6  False                0.41  False                  0.41  icecream or icecream sandwich th
    3  False                0.32  False                  0.32  tea oh tea
